In [11]:
import getpass
import os
import langchain

from langchain.schema.messages import HumanMessage, SystemMessage,AIMessage

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint="https://12205-m2hl4tqk-eastus.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-08-01-preview",
    azure_deployment="gpt-35-turbo",
    openai_api_version="2024-08-01-preview",
    api_key="d7f27353b3b3463bb02b2708df922f35",  
)

In [22]:
#消息列表
messages = [
    SystemMessage(content="用中文回复，并且字数控制在三十字以内"),
    HumanMessage(content="告诉我一个笑话"),
  
]
model.invoke(messages)

AIMessage(content='为什么小明的作业总是迟交？因为他总在等待灵感，但是灵感从来不会准时到达。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 39, 'total_tokens': 80, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-00226676-27f9-4a6a-aa7a-8f168541cf25-0', usage_metadata=

### 输出解析器

In [34]:
# 请注意，模型的响应是一个AIMessage。这包含一个字符串响应以及关于响应的其他元数据。我们通常可能只想处理字符串响应。我们可以通过使用简单的输出解析器来解析出这个响应。

# 我们首先导入简单的输出解析器。
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

#使用它的一种方法是单独使用它。例如，我们可以保存语言模型调用的结果，然后将其传递给解析器。
result = model.invoke(messages)
parser.invoke(result)



'为什么小鸟不会玩扑克牌？因为它们都会喊：“麻雀虽小，五脏俱全！”'

In [23]:
# 更常见的是，我们可以将模型与此输出解析器“链式”连接。这意味着在此链中，每次都会调用此输出解析器。
# 此链采用语言模型的输入类型（字符串或消息列表）并返回输出解析器的输出类型（字符串）。

# 我们可以使用 | 运算符轻松创建链。| 运算符在 LangChain 中用于将两个元素组合在一起。
chain=model|parser
result=model.invoke(messages)
parser.invoke(result)


'为什么小鸟会掉在地上？因为它不会飞！'

In [38]:
from langchain_core.runnables import Runnable
isinstance(model,Runnable)#显然 想要使用 invoke(调用一个运行接口)这个方法 必须是一个Runnable对象

True

# 提示词模板

In [43]:
#现在我们直接将消息列表传递给语言模型
#通常，它是由用户输入和应用逻辑的组合构建而成的。
#这个应用逻辑通常会将原始用户输入转换为准备传递给语言模型的消息列表。常见的
#转换包括添加系统消息或使用用户输入格式化模板


In [44]:
#language: 要翻译成的语言
#text: 要翻译的文本

In [45]:
#首先，让我们创建一个字符串，我们将格式化为系统消息：
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"

In [62]:
#接下来，我们可以创建 PromptTemplate。这将是 system_templ
#ate 和一个更简单的模板的组合，用于放置要翻译的文本
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [63]:
result=prompt_template.invoke({'language':'english','text':"what is your name"})
#分别替代对应的占位符
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into english:', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your name', additional_kwargs={}, response_metadata={})])

In [65]:
#我们可以看到它返回一个 ChatPromptValue，由两个消息组成。如果我们想直
#接访问这些消息，可以这样做：
result.to_messages()

[SystemMessage(content='Translate the following into english:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is your name', additional_kwargs={}, response_metadata={})]

# 使用 LCEL 连接组件

In [67]:
#我们现在可以使用管道 (|) 操作符将其与上面的模型和输出解析器结合起来：

In [70]:
chain = prompt_template|  model | parser

In [71]:
chain.invoke({"language": "italian", "text": "hi"})
#管道的链接有先后顺序

'ciao'

In [ ]:
!pip install numpy